In [1]:
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
from configbot import conkey, conkeysecret, atoken, atokensecret
import pandas as pd
import numpy as np

In [2]:
analyzer = SentimentIntensityAnalyzer()
auth = tweepy.OAuthHandler(conkey,conkeysecret)
auth.set_access_token(atoken,atokensecret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(),wait_on_rate_limit=True)

In [3]:
stock_data = pd.read_csv("StocksDow.csv")
stock_data.head()

,Symbol,Company
0,AXP,American Express Co
1,AAPL,Apple Inc
2,BA,Boeing Co
3,CAT,Caterpillar Inc
4,CSCO,Cisco Systems Inc


In [4]:
#pulled from whatever source we're using, set as lists for now, but can be changed
pull_dates = ["2018-04-06","2018-04-05","2018-04-04","2018-04-03","2018-04-02","2018-04-01","2018-03-31"]

#for dataframe
stock_name_daily = []
sentiments_daily = []
pull_dates_daily = []

prev_date = "2018-04-07"
#loop to check for every stock for every date and grab a sample of 100 tweets
for stocks in stock_data['Company']:
    for dates in pull_dates:
        stock_text = []
        sentiments = []
        tweets = api.search(q=stocks,count=100,since=dates,until=prev_date)
        prev_date = dates
        #run through 100 tweets for that day
        for public_tweets in tweets['statuses']:
            stock_text.append(public_tweets['text'])
            #vader sentiments
            sentiments.append(analyzer.polarity_scores(public_tweets['text'])['compound'])
            #append daily data to tables
        sentiments_daily.append(np.mean(sentiments))
        stock_name_daily.append(stocks)  
        pull_dates_daily.append(dates)
        
#Setting Up Dataframe                
twitter_df = pd.DataFrame({'Stock Name':stock_name_daily,
                            'Date':pull_dates_daily,
                            'Sentiments':sentiments_daily})
twitter_df.head()

C:\Users\conle\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\conle\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Date,Sentiments,Stock Name
0,2018-04-06,-0.000120,American Express Co
1,2018-04-05,0.005188,American Express Co
2,2018-04-04,0.067771,American Express Co
3,2018-04-03,0.256584,American Express Co
4,2018-04-02,0.092596,American Express Co


In [5]:
len(stock_name_daily)

210

In [6]:
len(pull_dates_daily)

210

In [7]:
len(sentiments_daily)

210

In [8]:
twitter_df.to_csv('stock_sentiments.csv')